In [9]:
using LinearAlgebra

# Autovalores e Autovetores de uma Matriz
Conteúdo referente às aulas 16 a 20

Veja também em:

https://replit.com/@erlonL/ProvaALC

# Diagonalização

Uma matriz de ordem $n$ é diagonalizável $\Leftrightarrow$ ela tiver $n$ autovetores $L.I.$

Desde que $$A = \begin{bmatrix} 1 & 1 \\ -2 & 4 \end{bmatrix} \text{ e } P^{-1}AP = \begin{bmatrix} 2 & 0 \\ 0 & 3 \end{bmatrix}$$
Então $A$ é diagonalizável.

Em que $P$ é uma matriz da forma $\begin{bmatrix} v^{(1)} & v^{(2)} \end{bmatrix}$ formada pelos autovetores de $A$.

In [10]:
function Auto(A)
    Autovalues = round.(eigen(A).values);
    Autovectors = eigen(A).vectors;
    return Autovalues, Autovectors
end

Auto (generic function with 1 method)

In [11]:
A = [3 0 1 ; 2 2 2 ; 4 2 5];
Ava, Avec = Auto(A)
P = Avec
round.((inv(P)*A*P) ; digits = 5)

3×3 Matrix{Float64}:
  1.0  -0.0  0.0
  0.0   2.0  0.0
 -0.0  -0.0  7.0

In [13]:
B = [2 1 0 ; 2 5 3 ; 0 1 6];
Ava, Avec = Auto(B)
P = Avec
round.((inv(P)*B*P) ; digits = 5) # = D
# P^-1 * A * P = D
# P é formada pelos autovetores de V

3×3 Matrix{Float64}:
  1.33754  -0.0      0.0
 -0.0       4.21809  0.0
  0.0       0.0      7.44437

# Métodos das Potências

Método da Potência => Determina o autovalor de maior módulo de uma matriz e o seu autovetor associado.

Método da Potência Inversa => Analogamente, determina o autovalor com menor módulo e seu correspondente autovetor.
- Consiste em calcular pelo _método das potências_ o autovalor com maior módulo de $A^{-1}$

In [14]:
function potencia(A, x, j, M)
    x0 = normalize(x,Inf)
    lambda =0.0
    for k=1:M
        y = A * x0
        lambda = y[j]/x0[j]
        x0 = normalize(y,Inf)
    end
    return lambda, x0
end

potencia (generic function with 1 method)

In [16]:
function potencia2(A, x, M)
    x0 = normalize(x,Inf)
    lambda =0.0
    p = 1
    while abs(x0[p]) < norm(x0, Inf)
        p = p+1
    end
    #@info("", p)
    for k=1:M
        y = A * x0
        lambda = y[p]/x0[p]
        p=1
        while abs(y[p]) < norm(y, Inf)
            p = p+1
        end
        #@info("", p)
        if y[p]==0
            lambda = 0.0
            return lambda, x0
        end
        x0 =normalize(y,Inf)
    end
    return lambda, x0
end

potencia2 (generic function with 1 method)

In [17]:
function potenciaINV(A, x, M)
    x0 = normalize(x,Inf)
    lambda =0.0
    p = 1
    while abs(x0[p]) < norm(x0, Inf)
        p = p+1
    end
    #@info("", p)
    for k=1:M
        y = A \x0
        lambda = y[p]/x0[p]
        p=1
        while abs(y[p]) < norm(y, Inf)
            p = p+1
        end
        #@info("", p)
        if y[p]==0
            lambda = 0.0
            return lambda, x0
        end
        x0 =normalize(y,Inf)
    end
    return 1/lambda, x0
end

potenciaINV (generic function with 1 method)

In [18]:
A = [3 0 1 ; 2 2 2 ; 4 2 5];
Ax0 = [1.0, 1.0, 1.0]
display(potencia(A, Ax0, 1, 100))
display(potenciaINV(A, Ax0, 100))

(7.0, [0.25, 0.5, 1.0])

(1.0, [0.5, 1.0, -1.0])

In [19]:
B = [2 1 0 ; 2 5 3 ; 0 1 6];
Bx0 = [1.0, 1.0, 1.0]
display(potencia(B, Bx0, 1, 100))
display(potenciaINV(B, Bx0, 100))

(7.444374823657817, [0.18367581814070388, 1.0, 0.6923410624588032])

(1.3375351956376103, [1.0, -0.6624648043623895, 0.14208467670202266])

# Decomposição por Valores Singulares (SVD)

Dada uma matriz $A$ de ordem $m \times n$:
- $A^TA$ é uma matriz quadrada, de ordem $n$;
- $A^TA$ é simétrica, então os seus autovalores são reais;
- $A^TA$ é positiva semi-definida, então os seus autovalores são não negativos;

Os valores singulares de uma matriz $A$ são denotados por
$$\sigma_1, \sigma_2, ..., \sigma_r$$
e são da forma:
$$\sigma_l = \sqrt{\lambda_l} > 0, \text{ para } l = 1, 2, ..., r.$$

- $v_i \in \mathbb{R}^n$ o autovetor ortonormal associado ao autovalor $\lambda_i$ para $i = 1, 2, ..., r$ de $A^TA$. Tal que:

$$V = \begin{bmatrix} v_1 & v_2 & ... & v_r\end{bmatrix}_{n \times r}$$

- $u_i \in \mathbb{R}^m$ tal que:

$$u_i = \frac{1}{\sigma_i}Av_i, \text{ para } i = 1, 2, ..., r.$$

$$U = \begin{bmatrix} u_1 & u_2 & ... & u_r\end{bmatrix}_{m \times r}$$

Também temos que:

$$\Sigma = \begin{bmatrix} \sigma_1 & & & \\ & \sigma_2 & & \\ & & \ddots & \\ & & & \sigma_r \end{bmatrix}$$

Logo, 

$$A = U\Sigma V^T$$

In [23]:
function SVD(A)
    n, m = size(A)

    Ava, Avec = Auto(A'*A)

    U = zeros(n,m)
    Σ = zeros(m, m)
    V = Avec

    for i = 1:m
        σi = sqrt(Ava[i])
        ui = (A * Avec[:,i])/σi
        U[:,i] = ui
        Σ[i,i] = σi
        # @info("σ$i = $σi")
        # @info("u$i = $ui")
        # @info("v$i = $(V[:,i])")
    end
    return U, Σ, V
end


SVD (generic function with 1 method)

In [24]:
A = [3 5 ; 4 0 ; 0 0]
Ava, Avec = Auto(A'*A)
σ1 = sqrt(Ava[1])
σ2 = sqrt(Ava[2])

u1 = (A * Avec[:,1])/σ1;
u2 = (A * Avec[:,2])/σ2;

U = [u1 u2]
Σ = [σ1 0 ; 0 σ2]
V = Avec

display(U*Σ*V')

3×2 Matrix{Float64}:
 3.0  5.0
 4.0  1.91844e-17
 0.0  0.0

In [25]:
S, V, D = SVD(A)
display(S*V*D')

3×2 Matrix{Float64}:
 3.0  5.0
 4.0  1.91844e-17
 0.0  0.0

In [27]:
A = [3 5 ; 5 -sqrt(2) ; -sqrt(2) -5];
S, V, D = SVD(A)

display(S*V*D')

3×2 Matrix{Float64}:
  3.0       5.0
  5.0      -1.41421
 -1.41421  -5.0